In [ ]:
from datetime import datetime

import time
import os
import sys
from pathlib import Path

import numpy as np
from scipy.stats import lognorm
import pandas as pd

from astropy import stats
import astropy.units as u
from astropy.io import fits

import matplotlib
import matplotlib.colors as mcolors

from matplotlib import style

import matplotlib.pyplot as plt

from mmtwfs.wfs import WFSFactory

from minicyclop.io import read_seeing_data

style.use('ggplot')

%load_ext autoreload
%autoreload 2
#%matplotlib widget

SMALLER_SIZE = 14
SMALL_SIZE = 16
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

In [ ]:
# instantiate all of the WFS systems...
wfs_keys = ['f9', 'newf9', 'f5', 'mmirs', 'binospec']
wfs_systems = {}
wfs_names = {}
for w in wfs_keys:
    wfs_systems[w] = WFSFactory(wfs=w)
    wfs_names[w] = wfs_systems[w].name

# give mmirs a default
wfs_systems['mmirs'].default_mode = 'mmirs1'

# map f9 to oldf9
wfs_systems['oldf9'] = wfs_systems['f9']

plt.close('all')

In [ ]:
rootdir = Path("./data")
# use find to make this csv list
with open(rootdir / "all_results.txt") as f:
    csvs = f.readlines()

# loop through the csv files and read them with pandas
frames = []
for csv in csvs:
    frames.append(pd.read_csv(rootdir / csv.rstrip()))

# now concat each frame into one big frame containing all of the data
data = pd.concat(frames)

In [ ]:
g = data.groupby('wfs')
g.wfs.describe()

In [ ]:
# re-calculate the seeing using updated spot FWHM for binospec. filter out any NaN values...
#fixed_list = []
#for wfskey, group in g:
#    wfs = wfs_systems[wfskey]
#    group = g.get_group(wfskey)
#    #group.apply(seeing, axis=1, args=(wfs,))
#    group['fixed_raw_seeing'], group['fixed_seeing'] = seeing(group, wfs)
#    fixed_list.append(group)
#fixed = pd.concat(fixed_list)
#fixed.rename(index=str, columns={"seeing": "orig_seeing", "raw seeing": "orig_raw_seeing"}, inplace=True)
#fixed.rename(index=str, columns={"fixed_seeing": "seeing", "fixed_raw_seeing": "raw seeing"}, inplace=True)
#data['seeing'][data['seeing'].apply(isinstance, args=(str,))] = 0.0
fixed = data[[isinstance(x, float) for x in data['seeing']]]
fixed['seeing'] = fixed['seeing'].astype(float)
fixed['vlt_seeing'] = fixed['vlt_seeing'].astype(float)
fixed['airmass'] = fixed['airmass'].astype(float)
fixed = fixed[np.isfinite(fixed['seeing'])]
fixed = fixed[np.isfinite(fixed['vlt_seeing'])]
fixed = fixed[fixed['fwhm'] > 0.]
fixed = fixed[fixed['vlt_seeing'] > 0.2]
fixed = fixed[fixed['vlt_seeing'] < 4.]
fixed = fixed[np.isfinite(fixed['airmass'])]
fixed = fixed[fixed['exptime'] >= 5.]

In [ ]:
# create a date-time index so we can group and analyze the data by timestamps
fixed = fixed.set_index(pd.DatetimeIndex(fixed['time'], name='ut'))

In [ ]:
fixed['seeing'].median(), fixed['vlt_seeing'].median()

In [ ]:
fixed.rename(columns={"seeing": "old_seeing", "vlt_seeing": "seeing", "raw_seeing": "old_raw_seeing", "raw_vlt_seeing": "raw_seeing"}, inplace=True)
fixed

In [ ]:
fixed.to_csv(rootdir / "wfs_seeing_2003_2025.csv")

In [ ]:
# make raw histogram of the zenith-corrected seeing column
#fixed.hist(column='seeing', bins=100, range=(0.0, 4.0), alpha=0.6)
with plt.style.context('ggplot'):
    fig = plt.figure(figsize=(8, 5))
    plt.hist(fixed['seeing'], bins=100, range=(0.0, 4.), alpha=0.6, label="no correction")
    plt.title("March 2003 through April 2025")
    plt.xlabel("Seeing (arcsec)")
    plt.ylabel("N")
plt.show()

In [ ]:
with plt.style.context('ggplot'):
    fig = plt.figure(figsize=(8, 5))
    medb = np.median(fixed['seeing'][fixed['wfs'] == 'binospec'])
    medm = np.median(fixed['seeing'][fixed['wfs'] == 'mmirs'])
    med5 = np.median(fixed['seeing'][fixed['wfs'] == 'f5'])
    med9 = np.median(fixed['seeing'][(fixed['wfs'] == 'oldf9') | (fixed['wfs'] == 'newf9')])
    plt.hist(
        fixed['seeing'][fixed['wfs'] == 'binospec'],
        density=True,
        bins=100,
        range=(0.0, 4.),
        alpha=0.6,
        label=f"Binospec: {medb: .2f}\""
    )
    plt.hist(
        fixed['seeing'][fixed['wfs'] == 'mmirs'],
        density=True,
        bins=100,
        range=(0.0, 4.),
        alpha=0.6,
        label=f"MMIRS: {medm: .2f}\""
    )
    plt.hist(
        fixed['seeing'][fixed['wfs'] == 'f5'],
        density=True,
        bins=100,
        range=(0.0, 4.),
        alpha=0.6,
        label=f"F/5: {med5: .2f}\""
    )
    plt.hist(
        fixed['seeing'][(fixed['wfs'] == 'oldf9') | (fixed['wfs'] == 'newf9')],
        density=True,
        bins=100,
        range=(0.0, 4.),
        alpha=0.6,
        label=f"F/9: {med9: .2f}\""
    )
    plt.legend()
    plt.title("Seeing vs Instrument (2003-2025)")
    plt.xlabel("Seeing (arcsec)")
    plt.ylabel("Number Density")
plt.savefig("all_bino_vs_mmirs_vs_f5_vs_f9.pdf")
plt.savefig("all_bino_vs_mmirs_vs_f5_vs_f9.png")
plt.show()

In [ ]:
with plt.style.context('ggplot'):
    fig = plt.figure(figsize=(8, 5))
    medb = np.median(fixed['exptime'][fixed['wfs'] == 'binospec'])
    medm = np.median(fixed['exptime'][fixed['wfs'] == 'mmirs'])
    med5 = np.median(fixed['exptime'][fixed['wfs'] == 'f5'])
    med9 = np.median(fixed['exptime'][(fixed['wfs'] == 'oldf9') | (fixed['wfs'] == 'newf9')])
    plt.hist(
        fixed['exptime'][fixed['wfs'] == 'binospec'],
        density=True,
        bins=100,
        range=(0.0, 120.),
        alpha=0.6,
        label=f"Binospec: {medb: .2f}\""
    )
    plt.hist(
        fixed['exptime'][fixed['wfs'] == 'mmirs'],
        density=True,
        bins=100,
        range=(0.0, 120.),
        alpha=0.6,
        label=f"MMIRS: {medm: .2f}\""
    )
    plt.hist(
        fixed['exptime'][fixed['wfs'] == 'f5'],
        density=True,
        bins=100,
        range=(0.0, 120.),
        alpha=0.6,
        label=f"F/5: {med5: .2f}\""
    )
    plt.hist(
        fixed['exptime'][(fixed['wfs'] == 'oldf9') | (fixed['wfs'] == 'newf9')],
        density=True,
        bins=100,
        range=(0.0, 120.),
        alpha=0.6,
        label=f"F/9: {med9: .2f}\""
    )
    plt.legend()
    plt.title("Exposure Time vs Instrument (2003-2025)")
    plt.xlabel("Exposure Time (s)")
    plt.ylabel("Number Density")
plt.savefig("all_exptime_bino_vs_mmirs_vs_f5_vs_f9.pdf")
plt.savefig("all_exptime_bino_vs_mmirs_vs_f5_vs_f9.png")
plt.show()

In [ ]:
with plt.style.context('ggplot'):
    fig, axs = plt.subplots(4, 2, sharex=True, sharey=True, figsize=(9, 12))
    fig.suptitle("Binospec vs MMIRS")
    fig.supylabel("Number Density")
    fig.supxlabel("Seeing (arcsec)")
    years = [str(y) for y in range(2018, 2026)]
    for ((i, year), ax) in zip(enumerate(years), axs.flat):
        year_df = fixed.loc[year]
        medb = np.median(year_df['seeing'][year_df['wfs'] == 'binospec'])
        medm = np.median(year_df['seeing'][year_df['wfs'] == 'mmirs'])
        ax.hist(
            year_df['seeing'][year_df['wfs'] == 'binospec'],
            density=True,
            bins=100,
            range=(0.0, 4.),
            alpha=0.6,
            label=f"Binospec: {medb: .2f}\""
        )
        ax.hist(
            year_df['seeing'][year_df['wfs'] == 'mmirs'],
            density=True,
            bins=100,
            range=(0.0, 4.),
            alpha=0.6,
            label=f"MMIRS: {medm: .2f}\""
        )
        ax.annotate(
            year,
            (0.75, 0.25),
            xycoords='axes fraction',
            ha='center',
            va='center',
            fontsize=18,
            color='darkgrey'
        )
        ax.legend()
plt.savefig("binospec_vs_mmirs_yearly.pdf")
plt.savefig("binospec_vs_mmirs_yearly.png")
plt.show()

In [ ]:
fixed['seeing'][fixed['wfs'] == 'binospec'].median(), fixed['seeing'][fixed['wfs'] == 'mmirs'].median()

In [ ]:
# fit a log-normal distribution to the seeing data
seeing = np.array(fixed['seeing'], dtype=float)
seeing = seeing[np.isfinite(seeing)]
sigma, loc, exp_mu = lognorm.fit(seeing)
print(sigma, loc, exp_mu)
x = np.arange(0.0, 4.0, 0.01)
p = lognorm.pdf(x, sigma, loc=loc, scale=exp_mu)
mu = np.log(exp_mu)
mode = np.exp(mu - sigma**2) + loc
median = np.median(seeing)
fit_median = exp_mu + loc

In [ ]:
# plot normalized histogram with the fit and the median/mode calculated from the fit.
fig = plt.figure(figsize=(8, 5))
with plt.style.context('ggplot'):
    plt.hist(fixed['seeing'], density=True, bins=100, range=(0.0, 4.0), label="hist", alpha=0.6)
    logp = plt.plot(x, p, label="logp")
    plt.xlabel("Seeing (arcsec)")
    plt.ylabel("Number Density")
    plt.title("March 2003 through January 2025")
    plt.legend(["median=%.2f\", mode=%.2f\"" % (fit_median, mode), "median=%.2f\"" % (
        median
    )])
    plt.savefig("all_hist.png")
    plt.savefig("all_hist.pdf")
plt.show()

In [ ]:
# now use the between_time() method to split the data into first and second halves of the night.
first_half = fixed.between_time(start_time='00:00', end_time='07:00')
second_half = fixed.between_time(start_time='07:00', end_time='14:00')
#first_half = months['December'].between_time(start_time='00:00', end_time='07:00')
#second_half = months['December'].between_time(start_time='07:00', end_time='14:00')
med_1st = np.nanmedian(first_half['seeing'])
med_2nd = np.nanmedian(second_half['seeing'])

In [ ]:
# plot the results and show that the seeing is better later in the night...
fig = plt.figure(figsize=(8, 5))
plt.hist(first_half['seeing'], bins=100, range=(0.0, 4.0), label="1st", alpha=0.6)
plt.hist(second_half['seeing'], bins=100, range=(0.0, 4.0), label="2nd", alpha=0.6)
plt.legend(["1st Half: %.2f\"" % med_1st, "2nd Half: %.2f\"" % med_2nd])
plt.xlabel("Seeing (arcsec)")
plt.ylabel("N")
#plt.title("Data and median seeing values grouped by half of the night")
plt.savefig("all_1st2nd.png")
plt.savefig("all_1st2nd.pdf")
plt.show()

In [ ]:
import matplotlib.dates as mdates

# make a new data frame that resamples the WFS data daily into the median, min, and max seeing for each day.
#fixed = fixed["2018-06-30":"2018-10-01"]
fig, ax = plt.subplots(figsize=(10, 6))
monthly = pd.DataFrame()
monthly['seeing'] = fixed.seeing.resample('Y', label='left').median()
monthly['max'] = fixed.seeing.resample('Y', label='left').max()
monthly['min'] = fixed.seeing.resample('Y', label='left').min()
lowb = monthly['seeing'] - monthly['min']
upb = monthly['max'] - monthly['seeing']
plt.errorbar(monthly.index, monthly['seeing'], yerr=[lowb, upb], fmt='o')
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
fig.autofmt_xdate()
plt.ylabel("Seeing (arcsec)")
#plt.title("Nightly median seeing")
plt.savefig("all_nightly.png")
plt.savefig("all_nightly.pdf")
plt.show()

In [ ]:
year_list = [str(y) for y in range(2003, 2026)]
data = [np.array(fixed.loc[month]['seeing']) for month in year_list]
ellip_data = [np.array(fixed.loc[month]['ellipticity']) for month in year_list]
clean_data = {}
clean_ellip = {}
for d, arr in zip(year_list, data):
    if len(arr) > 0:
        clean_data[d] = arr[np.isfinite(arr)]
for d, arr in zip(year_list, ellip_data):
    if len(arr) > 0:
        clean_ellip[d] = arr[np.isfinite(arr)]
year_labels = [datetime.strptime(d, "%Y").date() for d in clean_data.keys()]
sizes = [len(clean_data[y]) for y in clean_data]
year_list

In [ ]:
first_meds = []
sec_meds = []
for y in clean_data:
    year = fixed.loc[y]
    first_half = year.between_time(start_time='00:00', end_time='07:00')
    second_half = year.between_time(start_time='07:00', end_time='14:00')
    med_1st = np.nanmedian(first_half['seeing'])
    med_2nd = np.nanmedian(second_half['seeing'])
    first_meds.append(med_1st)
    sec_meds.append(med_2nd)
    print(f"{y}: {med_1st:.2f} {med_2nd:.2f}")

In [ ]:
with plt.style.context('ggplot'):
    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('font', family='serif')
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALLER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALLER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title
    fig, ax = plt.subplots(4, figsize=(11,15), sharex=True)
    ax[0].bar(mdates.date2num(year_labels), sizes, width=366, alpha=0.6)
    ax[0].set_ylabel("# of Seeing Measurements")
    ax[1].violinplot(clean_data.values(), mdates.date2num(year_labels), points=100, widths=365,
                showextrema=False, showmedians=True, bw_method='silverman')
    ax[1].set_ylim(0.0, 3.0)
    ax[1].set_ylabel("Seeing (arcsec)")
    ax[2].violinplot(clean_ellip.values(), mdates.date2num(year_labels), points=100, widths=365,
                showextrema=False, showmedians=True, bw_method='silverman')
    ax[2].set_ylim(0.0, 0.5)
    ax[2].set_ylabel("Ellipticity")
    ax[3].scatter(mdates.date2num(year_labels), first_meds, label="1st Half")
    ax[3].scatter(mdates.date2num(year_labels), sec_meds, label="2nd Half")
    ax[3].legend(frameon=True, fancybox=True)
    ax[3].set_ylim(0.65, 1.25)
    ax[3].set_ylabel("Median Seeing (arcsec)")
    ax[3].xaxis.set_major_locator(mdates.YearLocator())
    ax[3].xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
    fig.autofmt_xdate()
    fig.tight_layout()
    plt.savefig("all_violin.png")
    plt.savefig("all_violin.pdf")
    plt.show()

In [ ]:
[len(clean_data[y]) for y in clean_data]

In [ ]:
diff = fixed['osst'] - fixed['outt']
trim_seeing = fixed['seeing'][np.abs(diff) < 100]
diff = diff[np.abs(diff) < 100]

In [ ]:
with plt.style.context('ggplot'):
    fig, ax = plt.subplots()
    ax.hist(diff, bins=100, range=(-10, 10))
plt.show()

In [ ]:
temps = list(range(-3,6))
t_meds = []
for t in temps:
    seeing_slice = trim_seeing[(diff >= t-0.5) & (diff <= t+0.5)]
    t_meds.append(np.median(seeing_slice))
t_meds, temps

In [ ]:
with plt.style.context('default'):
    fig, ax = plt.subplots()
    plt.rc('font', size=SMALLER_SIZE)          # controls default text sizes
    plt.rc('font', family='serif')
    h = ax.hist2d(trim_seeing, diff, bins=100, cmap='turbo', norm=mcolors.PowerNorm(0.3))
    fig.supxlabel("Seeing (arcsec)")
    fig.supylabel("$T_{OSS} - T_{Ambient}$ (C)")
    cb = fig.colorbar(h[-1])
    cb.set_label("Counts/bin")
    ax.set_ybound(-6, 6)
    ax.set_xbound(0.4, 3.0)
    ax.scatter(t_meds, temps, color='w')
    ax.tick_params(axis='both', labelsize=12)

plt.savefig("seeing_tempdiff.png")
plt.savefig("seeing_tempdiff.pdf")
plt.show()

In [ ]:
trimmed = fixed[np.isfinite(fixed['seeing'].astype(float)) & np.isfinite(fixed['el'].astype(float))]
trim_el = trimmed['el'].astype(float)
el_seeing = trimmed['seeing'].astype(float)
els = [25, 35, 45, 55, 65, 75, 85]
e_meds = []
for el in els:
    seeing_slice = el_seeing[(trim_el >= el-5) & (trim_el <= el+5)]
    e_meds.append(np.median(seeing_slice))
with plt.style.context('default'):
    fig, ax = plt.subplots()
    plt.rc('font', size=SMALLER_SIZE)          # controls default text sizes
    plt.rc('font', family='serif')
    h = ax.hist2d(el_seeing, trim_el, bins=100, cmap='turbo', norm=mcolors.PowerNorm(0.3))
    fig.supxlabel("Seeing (arcsec)")
    fig.supylabel("Elevation")
    cb = fig.colorbar(h[-1])
    cb.set_label("Counts/bin")
    plt.scatter(e_meds, els, color='w')
    ax.set_xbound(0.4, 3.0)
    ax.tick_params(axis='both', labelsize=12)

plt.savefig("seeing_elevation.png")
plt.savefig("seeing_elevation.pdf")
plt.show()

In [ ]:
trimmed = fixed[np.isfinite(fixed['seeing'].astype(float)) & np.isfinite(fixed['az'].astype(float))]
trim_az = trimmed['az'].astype(float)
trim_az[trim_az < 0] += 360.
az_seeing = trimmed['seeing'].astype(float)
azs = [15, 45, 75, 105, 135, 165, 195, 225, 255, 285, 315, 345]
a_meds = []
for az in azs:
    seeing_slice = az_seeing[(trim_az >= az-15) & (trim_az <= az+15)]
    a_meds.append(np.median(seeing_slice))
with plt.style.context('default'):
    fig, ax = plt.subplots()
    plt.rc('font', size=SMALLER_SIZE)          # controls default text sizes
    plt.rc('font', family='serif')
    h = ax.hist2d(az_seeing, trim_az, bins=100, cmap='turbo', norm=mcolors.PowerNorm(0.3))
    fig.supxlabel("Seeing (arcsec)")
    fig.supylabel("Azimuth")
    cb = fig.colorbar(h[-1])
    cb.set_label("Counts/bin")
    ax.scatter(a_meds, azs, color='w')
    ax.set_xbound(0.4, 3.0)
    ax.tick_params(axis='both', labelsize=12)

plt.savefig("seeing_azimuth.png")
plt.savefig("seeing_azimuth.pdf")
plt.show()

In [ ]:
trimmed = fixed[np.isfinite(fixed['seeing'].astype(float)) & np.isfinite(fixed['outt'].astype(float))]
trim_outt = trimmed['outt'].astype(float)
outt_seeing = trimmed['seeing'].astype(float)
temps = list(range(-5, 25, 5))
temps_meds = []
for t in temps:
    seeing_slice = outt_seeing[(trim_outt >= t-2.5) & (trim_outt <= t+2.5)]
    temps_meds.append(np.median(seeing_slice))
with plt.style.context('default'):
    fig, ax = plt.subplots()
    plt.rc('font', size=SMALLER_SIZE)          # controls default text sizes
    plt.rc('font', family='serif')
    h = ax.hist2d(outt_seeing, trim_outt, bins=100, cmap='turbo', norm=mcolors.PowerNorm(0.3))
    fig.supxlabel("Seeing (arcsec)")
    fig.supylabel("$T_{Ambient}$ (C)")
    cb = fig.colorbar(h[-1])
    cb.set_label("Counts/bin")
    ax.scatter(temps_meds, temps, color='w')
    ax.set_xbound(0.4, 3.0)
    ax.tick_params(axis='both', labelsize=12)

plt.savefig("seeing_ambient.pdf")
plt.savefig("seeing_ambient.png")
plt.show()

In [ ]:
from minicyclop.io import read_seeing_data
cyc = read_seeing_data(Path.home() / "MMT/minicyclop/data/MiniCyclop/Data/Seeing_Data.txt")
cyc.drop('UT', axis=1, inplace=True)
cyc

In [ ]:
fixed_overlap = fixed[fixed.index >= "2022-06-08"]

In [ ]:
with plt.style.context('ggplot'):
    fig, ax = plt.subplots()
    med_wfs = np.median(fixed_overlap['seeing'])
    med_cyc = np.median(cyc['seeing'])
    w = ax.hist(
        fixed_overlap['seeing'],
        density=True,
        bins=100,
        range=(0.0, 4.0),
        label=f"WFS: {med_wfs: .2f}\" (N={len(fixed_overlap)})",
        alpha=0.6
    )
    c = ax.hist(
        cyc['seeing'],
        density=True,
        bins=100,
        range=(0.0, 4.0),
        label=f"Cyclop: {med_cyc: .2f}\" (N={len(cyc)})",
        alpha=0.6
    )
    fig.supxlabel("Seeing (arcsec)")
    fig.supylabel("Number Density")
    fig.suptitle("WFS vs. Cyclop Seeing Monitor (6/8/2022 onward)")
    ax.legend()
plt.savefig("wfs_cyclop.pdf")
plt.savefig("wfs_cyclop.png")
plt.show()

In [ ]:
f9 = fixed_overlap[fixed_overlap['wfs'] == 'newf9']
bino = fixed_overlap[fixed_overlap['wfs'] == 'binospec']
hecto = fixed_overlap[fixed_overlap['wfs'] == 'f5']
mmirs = fixed_overlap[fixed_overlap['wfs'] == 'mmirs']

In [ ]:
all_f9 = fixed[fixed['wfs'] == 'newf9']
all_f9_dates = sorted(list(set(all_f9.index.strftime('%Y-%m-%d'))))
with open("f9_dates.txt", "w") as f:
    for d in all_f9_dates:
        f.write(f"{d.replace("-", "")}\n")

In [ ]:
f9_dates = sorted(list(set(f9.index.strftime('%Y-%m-%d'))))
bino_dates = sorted(list(set(bino.index.strftime('%Y-%m-%d'))))
hecto_dates = sorted(list(set(hecto.index.strftime('%Y-%m-%d'))))
mmirs_dates = sorted(list(set(mmirs.index.strftime('%Y-%m-%d'))))
bino_dates, hecto_dates, f9_dates, mmirs_dates

In [ ]:
cyc_f9 = np.hstack([np.array(cyc.loc[date]['seeing']) for date in f9_dates])
cyc_bino = np.hstack([np.array(cyc.loc[date]['seeing']) for date in bino_dates])
cyc_hecto = np.hstack([np.array(cyc.loc[date]['seeing']) for date in hecto_dates])
cyc_mmirs = np.hstack([np.array(cyc.loc[date]['seeing']) for date in mmirs_dates])

In [ ]:
with plt.style.context('ggplot', after_reset=False):
    fig, ax = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
    ax = ax.flat
    fig.subplots_adjust(hspace=0)

    bin_type = 40

    ax[0].hist(bino['seeing'], bins=bin_type, histtype='stepfilled', alpha=0.6, density=True)
    ax[0].hist(cyc_bino, bins=bin_type, histtype='stepfilled', alpha=0.6, density=True)
    ax[0].legend([f"Binospec: {np.median(bino['seeing']):.2f}", f"Cyclop: {np.median(cyc_bino):.2f}"])
    ax[0].set_ylabel("Probaility Density")
    ax[0].set_xlim(0, 4)

    ax[1].hist(hecto['seeing'], bins=bin_type, histtype='stepfilled', alpha=0.6, density=True)
    ax[1].hist(cyc_hecto, bins=bin_type, histtype='stepfilled', alpha=0.6, density=True)
    ax[1].legend([f"Hecto: {np.median(hecto['seeing']):.2f}", f"Cyclop: {np.median(cyc_hecto):.2f}"])
    ax[1].set_xlim(0, 4)

    ax[2].hist(f9['seeing'], bins=bin_type, histtype='stepfilled', alpha=0.6, density=True)
    ax[2].hist(cyc_f9, bins=bin_type, histtype='stepfilled', alpha=0.6, density=True)
    ax[2].legend([f"F/9: {np.median(f9['seeing']):.2f}", f"Cyclop: {np.median(cyc_f9):.2f}"])
    ax[2].set_xlim(0, 4)
    ax[2].set_xlabel("Seeing (arcsec)")
    ax[2].set_ylabel("Probaility Density")

    ax[3].hist(mmirs['seeing'], bins=bin_type, histtype='stepfilled', alpha=0.6, density=True)
    ax[3].hist(cyc_mmirs, bins=bin_type, histtype='stepfilled', alpha=0.6, density=True)
    ax[3].legend([f"MMIRS: {np.median(mmirs['seeing']):.2f}", f"Cyclop: {np.median(cyc_mmirs):.2f}"])
    ax[3].set_xlim(0, 4)
    ax[3].set_xlabel("Seeing (arcsec)")

    plt.tight_layout()
    plt.savefig("all_cyclop_vs_inst.png")
    plt.show()

In [ ]:
cyc.to_csv(rootdir / "cyclop_seeing_2003_2025.csv")